In [6]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

In [7]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')

def get_eye_info(i): return ejson(i, 'appleLeftEye.json'), ejson(i, 'appleRightEye.json')
def get_face_info(i): return ejson(i, 'appleFace.json')
def get_facegrid(i): return ejson(i, 'faceGrid.json')

def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords
def orientation_data(p):
    sdata = screen_data(p)
    return sdata['Orientation']

In [8]:
# dset_path = Path('/media/brennan/Data/ml/gazecapture/')
dset_path = Path('../gazecapture/')

# Extract File Names For Keras Generator

In [9]:
test_size = 0.1

In [19]:
invalid_cases = ['02984',
                 '02032',
                 '01030',
                 '01152',
                 '00325',
                 '02622',
                 '00827',
                 '02119',
                 '00927',
                 '03277',
                 '02190',
                 '01282',
                 '00808',
                 '01474',
                 '01805',
                 '00207',
                 '00666',
                 '01066',
                 '01243',
                 '01269',
                 '01661',
                 '02048',
                 '01266',
                 '00619',
                 '00779',
                 '01090',
                 '03059',
                 '01432',
                 '01225',
                 '03060',
                 '02781',
                 '00138',
                 '02526',
                 '01353',
                 '02967',
                 '02093',
                 '02165']

In [18]:
def extract_data(cases, accepted_o=[1, 2, 3, 4]):
    fnames = []
    XCam, YCam = [], []
    FaceH, FaceW, FaceX, FaceY = [], [], [], []
    IsValid = []
    
    i = 0
    for case in cases:
        if not (case in invalid_cases):
            FRAME_N = frame_data(case)
            O = orientation_data(case)
            XCAM, YCAM = coordinate_data(case)
    #         LE, RE = get_eye_info(case)
            FACE = get_face_info(case)

            for frame_n, o, xcam, ycam, \
                fh, fw, fx, fy, valid in zip(FRAME_N, O, XCAM, YCAM, 
                                     FACE['H'], FACE['W'], FACE['X'], FACE['Y'], FACE['IsValid']):
                if o in accepted_o and valid == 1:
                    fnames.append('{}/frames/{}'.format(case.name, frame_n))
                    XCam.append(xcam)
                    YCam.append(ycam)
                    IsValid.append(valid)
                    FaceH.append(fh)
                    FaceW.append(fw)
                    FaceX.append(fx)
                    FaceY.append(fy)

    # package to dataframe
    df = pd.DataFrame(data={'file_names': fnames, 
                            'XCam': XCam,
                            'YCam': YCam,
                            'IsValid': IsValid,
                            'FaceH': FaceH, 
                            'FaceW': FaceW,
                            'FaceX': FaceX,
                            'FaceY': FaceY})        
    
    return df

# Extract all case names first to split => faces it hasnt seen before
train_cases, test_cases = train_test_split(list(dset_path.iterdir()), test_size=test_size)
len(train_cases), len(test_cases)

(1326, 148)

# Extract all Images

In [23]:
# create dataframe from data
# ALL PHOTOS split into train / test sets
train_df = extract_data(train_cases)
test_df = extract_data(test_cases)

train_df.to_csv('train-df.csv')
test_df.to_csv('test-df.csv')

# Extract ONLY Portrait Images

In [19]:
train_df = extract_data(train_cases, accepted_o=[1])
test_df = extract_data(test_cases, accepted_o=[1])
train_df.to_csv('portrait-data/traindf.csv')
test_df.to_csv('portrait-data/testdf.csv')

In [30]:
len(test_df), len(train_df)

(82893, 693269)

In [31]:
len(test_df) / (len(test_df) + len(train_df)), len(train_df) / (len(test_df) + len(train_df))

(0.10679858070866649, 0.8932014192913336)

# Extract & Split LandScape Images

In [20]:
train_df = extract_data(train_cases, accepted_o=[3])
test_df = extract_data(test_cases, accepted_o=[3])
train_df.to_csv('landscape-r-data/traindf.csv')
test_df.to_csv('landscape-r-data/testdf.csv')

In [21]:
train_df = extract_data(train_cases, accepted_o=[4])
test_df = extract_data(test_cases, accepted_o=[4])
train_df.to_csv('landscape-l-data/traindf.csv')
test_df.to_csv('landscape-l-data/testdf.csv')

## Stage 1 & 2 Split

In [ ]:
# train_df_sample = train_df.sample(frac=0.5)
# strain, sval = train_test_split(train_df_sample, test_size=0.1)
# strain.to_csv('traindf_sample.csv'), sval.to_csv('valdf_sample.csv')

## Update Prev Stage Dataset

In [23]:
# old_df = pd.read_csv('portrait-traindf.csv')
# file_names = old_df['file_names'].values
# cases_to_update = []

# for fn in file_names:
#     case = fn[:fn.index('/')]
#     if not (dset_path / case in cases_to_update):
#         cases_to_update.append(dset_path / case)

# new_train_df = extract_data(cases_to_update, accepted_o=[1])
# strain, sval = train_test_split(new_train_df, test_size=0.1)
# strain.to_csv('portrait-traindf.csv'), sval.to_csv('portrait-valdf.csv')

(None, None)

# Export to hdf5
Not Used

In [10]:
# # Extracting and storing X values
# def extract_photo(f, case): return get_frame(case, f)

# def extract(case):
#     F = frame_data(case) # list of file names correlated to camera images 
#     with Pool(6) as p:  # multi core usale
#         x = p.map(functools.partial(extract_photo, case=case), F)
#     return x

# def extract_to_hdf5(cname, bs, extract_fcn): # bs dsets per hp5y file
#     f = h5py.File('{}-0.hdf5'.format(cname), 'w') 
#     for i, case in enumerate(list(dset_path.iterdir())):
#         cn = case.name
#         edata = extract_fcn(case)
        
#         dset = f.create_dataset('dset'+cn, (len(edata), 224, 224, 3))
#         dset[...] = edata

#         if i % bs == 0 and i!= 0:
#             f.close()
#             f = h5py.File('{}-{}.hdf5'.format(cname, i//bs), 'w')
# extract_to_hdf5('X', 500, extract)

# Extract Split Which Was Used In The Paper
http://gazecapture.csail.mit.edu/cvpr2016_gazecapture.pdf

In [30]:
class Dataset:
    def __init__(self):
        self.fnames = []
        self.XCam = []
        self.YCam = []
        self.IsValid = []
        self.FaceH, self.FaceW, self.FaceX, self.FaceY = [], [], [], []
    
    def append(self, fname, xcam, ycam, valid, fh, fw, fx, fy):
        self.fnames.append(fname)
        self.XCam.append(xcam)
        self.YCam.append(ycam)
        self.IsValid.append(valid)
        self.FaceH.append(fh)
        self.FaceW.append(fw)
        self.FaceX.append(fx)
        self.FaceY.append(fy)

In [31]:
def extract_split(cases, accepted_o=[1, 2, 3, 4]):
    train, val, test = Dataset(), Dataset(), Dataset()
    
    i = 0
    for case in cases:
        if not (case in invalid_cases):
            FRAME_N = frame_data(case)
            O = orientation_data(case)
            XCAM, YCAM = coordinate_data(case)
            dset = info_data(case)['Dataset']
            FACE = get_face_info(case)
            
            if dset == 'train': dataset = train
            elif dset == 'val': dataset = val
            elif dset == 'test': dataset = test

            for frame_n, o, xcam, ycam, \
                fh, fw, fx, fy, valid in zip(FRAME_N, O, XCAM, YCAM, 
                                     FACE['H'], FACE['W'], FACE['X'], FACE['Y'], FACE['IsValid']):
                if o in accepted_o and valid == 1:
                    dataset.append('{}/frames/{}'.format(case.name, frame_n), 
                                   xcam, 
                                   ycam,
                                   valid,
                                   fh, fw, fx, fy)
    
    dset_df = []
    for dataset in [train, test, val]:
        # package to dataframe
        df = pd.DataFrame(data={'file_names': dataset.fnames, 
                                'XCam': dataset.XCam,
                                'YCam': dataset.YCam,
                                'IsValid': dataset.IsValid,
                                'FaceH': dataset.FaceH, 
                                'FaceW': dataset.FaceW,
                                'FaceX': dataset.FaceX,
                                'FaceY': dataset.FaceY})        
        dset_df.append(df)

    return dset_df[0], dset_df[1], dset_df[2]

In [32]:
traindf, valdf, testdf = extract_split(list(dset_path.iterdir()), accepted_o=[1])
traindf.to_csv('papersplit/portrait/traindf.csv')
valdf.to_csv('papersplit/portrait/valdf.csv')
testdf.to_csv('papersplit/portrait/testdf.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../gazecapture/02942/frames.json'